In [8]:
def affine_gap_alignment(seq1, seq2, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty):
    n = len(seq1)
    m = len(seq2)
    
    def create_M():
        M = [[0 for _ in range(n + 1)] for _ in range(m + 1)]

        M[0] = [open_gap_penalty + i * continue_gap_penalty for i in range(n + 1)]

        for j in range(m + 1):
            M[j][0] = open_gap_penalty + j * continue_gap_penalty

        M[0][0] = 0
        
        return M
    
    M_main = create_M()
    M_low = create_M()
    M_up = create_M()

    for i in range(1, m + 1):  # seq2
        for j in range(1, n + 1):  # seq1        
            def seq_match(a, b):
                if a == b:
                    return weight_match
                else:
                    return weight_mismatch
            
            s1_up = M_up[i-1][j] + continue_gap_penalty
            s2_up = M_main[i-1][j] + open_gap_penalty + continue_gap_penalty
            
            M_up[i][j] = max(s1_up, s2_up)
            
            s1_low = M_low[i][j-1] + continue_gap_penalty
            s2_low = M_main[i][j-1] + open_gap_penalty + continue_gap_penalty
            
            M_low[i][j] = max(s1_low, s2_low)
            
            s1 = M_main[i-1][j-1] + seq_match(seq1[j - 1], seq2[i - 1])
            s2 = M_up[i][j]
            s3 = M_low[i][j]
            
            s = max(max(s1, s2), s3)
            M_main[i][j] = s
    
    seq_new1 = ''
    seq_new2 = ''

    cur_i = m
    cur_j = n

    while cur_i > 0 or cur_j > 0:
        if M_main[cur_i - 1][cur_j - 1] >= M_main[cur_i][cur_j - 1] and M_main[cur_i - 1][cur_j - 1] >= M_main[cur_i - 1][cur_j]:
            seq_new1 = seq1[cur_j - 1] + seq_new1
            seq_new2 = seq2[cur_i - 1] + seq_new2
            cur_i = cur_i - 1
            cur_j = cur_j - 1
        elif M_main[cur_i][cur_j - 1] > M_main[cur_i - 1][cur_j - 1] and M_main[cur_i][cur_j - 1] > M_main[cur_i - 1][cur_j]:
            seq_new1 = seq1[cur_j - 1] + seq_new1
            seq_new2 = '_' + seq_new2
            cur_j = cur_j - 1
        elif M_main[cur_i - 1][cur_j] > M_main[cur_i - 1][cur_j - 1] and M_main[cur_i - 1][cur_j] > M_main[cur_i][cur_j - 1]:
            seq_new1 = '_' + seq_new1
            seq_new2 = seq2[cur_i - 1] + seq_new2
            cur_i = cur_i - 1

    print(seq_new1, seq_new2, sep='\n')

Сравним два варианта штрафов за открытие и продолжение гэпа

In [11]:
sequence1, sequence2 = 'actgagggacg agctactac'.split()

affine_gap_alignment(sequence1, sequence2, 3, -3, -2, -1)

a_ctgagggac___g
agct_a____ctac_


In [12]:
affine_gap_alignment(sequence1, sequence2, 3, -3, 0, -2)

a_ctgagggacg
agct_a_ctac_
